# Imports

In [ ]:
import subprocess
import json
from pathlib import Path
from glob2 import glob
import os
from multiprocessing import Pool
from datetime import datetime
from uuid import uuid4

# Functions

In [ ]:
def runFullParse (inputPiece, iterations=10):
    (corpus, piece) = inputPiece
    jsonPath = "outputs/"+corpus+"/"+piece+".json"
    cmd = ["stack","run","fullParse","--","-n",str(iterations), corpus, piece, "All"] 
    print("Running command: " + (" ".join(cmd)))
    print("Expecting results in " + jsonPath)
    
    #res = subprocess.run(cmd, cwd="..")
    #res = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd="..")
    res = subprocess.run(cmd,stderr=subprocess.PIPE, cwd="..")

    if res.returncode != 0:
        print("Error in subprocess:")
        print(res.stderr)
        return;
    else:
        f = open(jsonPath)
        results = json.load(f)
        f.close()
        return results
     
def get_corpus_pieces(corpus):
    return sorted (os.path.basename(d).split(".")[0] for d in glob("inputs/slices/"+corpus+"/*.csv"))

def get_corpi():
    return [os.path.basename(d) for d in glob("inputs/slices/*")][1:]

def run_experiment():
    corpi = get_corpi()
    for corpus in corpi:
        pieces = get_corpus_pieces(corpus)
        for piece in pieces:
            runFullParse((corpus, piece))

In [ ]:
get_corpi()

In [ ]:
def run_Experiment(commands,name,threads):
    id = datetime.now().strftime('%Y%m-%d%H-%M%S_') + name

    commands = [ ["stack","run","fullParse","--","-n",str(1), "-id", str(id)] + command for command in commands]
    with Pool(threads) as p:
        p.map(run_command, commands)


In [ ]:
def run_command(command):
    print("\nRunning command: " + (" ".join(command)))    
    #res = subprocess.run(cmd, cwd="..")
    #res = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd="..")
    res = subprocess.run(command,stderr=subprocess.PIPE, cwd="..")

    if res.returncode != 0:
        print("Error in subprocess:")
        print(res.stderr)
        return;


In [ ]:
def full_experiment(threads=20):
    commands = []
    corpi = get_corpi()
    for corpus in corpi:
        pieces = get_corpus_pieces(corpus)
        for piece in pieces:
            for algorithm in ["RandomWalk", "RandomSample", "RandomReduction","BeamSearch 10"]:
                commands.append([ corpus, piece, algorithm])

    run_Experiment(commands, "baseline", threads)

In [ ]:
def baseline_experiment(threads=20):
    commands = []
    corpi = get_corpi()
    for corpus in corpi:
        pieces = get_corpus_pieces(corpus)
        for piece in pieces:
            for algorithm in ["RandomWalk", "RandomSample", "RandomReduction"]:
                commands.append([ corpus, piece, algorithm])

    run_Experiment(commands, "baseline", threads)


In [ ]:
def beamwidth_experiment(threads=10):
    commands = []
    corpi = get_corpi()
    for corpus in corpi:
        pieces = get_corpus_pieces(corpus)
        for piece in pieces:
            for w in range (1,20):
                algo = "BeamSearch " + str(w)
                commands.append([ corpus, piece, algo])

    run_Experiment(commands, "beamwidth-beamsearch", threads)

In [ ]:
def stochastic_res_experiment(threads=10):
    commands = []
    corpi = get_corpi()
    for corpus in corpi:
        pieces = get_corpus_pieces(corpus)
        for piece in pieces:
            for w in range (2,30,3):
                for r in range(200,3000,400):
                    algo = "StochasticBeamSearch {} {}".format(w,r)
                    commands.append([ corpus, piece, algo])

    run_Experiment(commands, "stochastic-all-params", threads)

In [ ]:
print("\"io")

# Run Experiments

In [ ]:
if __name__ == '__main__':
    #algo_experiment()
    stochastic_res_experiment(20)

In [ ]:
if __name__ == '__main__':
    algo_experiment()
    #beamwidth_experiment()

In [ ]:
print("hmm")